<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-03 13:29:38
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: 2.68 L (1.97%)
Current PnL: -30.46 L (-19.42%)
CY Booked + Current PnL: -15.72 L (-10.02%)
-------------------
Total profit:  1.46 L
Total loss:  -31.91 L
-------------------
Total Booked + Current PnL: 11.26 L (8.67%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.6%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.97 L (69.02%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.26,4.41,14.19,19.23,22293.0,6642.0,157102.0,484.83,22.81,52.0,L-LC,12.49,182.0,0.30,1.13,26.63,XY25,ATH,MINING
50,MASFIN,1.80,-3.74,26.78,22.05,25259.0,-3660.0,94320.0,398.61,-16.89,50.0,H-SC,3.47,168.0,-0.14,0.68,37.10,XR,ATH,FINANCE
77,TTKPRESTIG,0.63,-27.00,37.12,0.09,27307.0,-27214.0,73563.0,770.00,70.76,35.0,M-SC,7.11,253.0,-1.00,0.53,0.63,OX40N,NTT,DURABLES
33,HCLTECH,1.44,10.39,12.18,23.83,32528.0,25133.0,267057.0,1908.19,15.30,57.0,X-LC,7.65,13.0,0.77,1.92,28.13,X40,ATH,IT
51,MEDANTA,-2.54,-7.22,33.73,24.08,39730.0,-9163.0,117787.0,1486.00,-14.79,48.0,X-SC,10.32,91.0,-0.23,0.85,7.88,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,11.52,-23.12,109.12,60.77,153330.0,-42260.0,140515.0,521.0,61.98,64.0,H-SC,4.46,158.0,-0.28,1.01,27.39,XR,NTT,JEWELLERY
76,TRIDENT,11.44,-23.05,67.01,28.51,47569.0,-21266.0,70988.0,48.0,-8.99,67.0,M-SC,10.74,226.0,-0.45,0.51,21.47,XR,NTT,TEXTILES
46,JIOFIN,8.64,-14.70,45.93,24.48,104753.0,-39293.0,228072.0,387.0,-16.12,43.0,X-LC,8.51,37.0,-0.38,1.64,32.20,XY24,BTT,FINANCE
5,ANGELONE,7.86,6.06,17.10,24.20,55361.0,18500.0,323750.0,3033.0,50.86,54.0,X-SC,1.99,99.0,0.33,2.33,35.59,X40N,NTT,FINANCE
56,RAJOOENG,6.92,-37.74,124.22,39.61,79277.0,-38680.0,63820.0,143.1,-57.95,46.0,H-SC,28.26,136.0,-0.49,0.46,6.92,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,-4.84,-10.36,51.36,35.67,145406.0,-32730.0,283112.0,671.64,-21.08,29.0,X-LC,1.16,5.0,-0.23,2.04,2.06,X40N,ATH,FMCG
45,JCHAC,-3.77,-42.19,72.96,-0.01,53907.0,-53919.0,73886.0,2282.00,15030.43,39.0,M-SC,15.50,235.0,-1.00,0.53,2.44,OX40N,BTT,AC
48,KANSAINER,-2.59,-23.87,49.06,13.47,100715.0,-64377.0,205290.0,340.00,-68.70,46.0,H-SC,5.12,159.0,-0.64,1.48,4.38,XY24,NTT,PAINTS
51,MEDANTA,-2.54,-7.22,33.73,24.08,39730.0,-9163.0,117787.0,1486.00,-14.79,48.0,X-SC,10.32,91.0,-0.23,0.85,7.88,XY24,NTT,HEALTHCARE
2,ABBOTINDIA,-1.59,-9.79,30.89,18.07,49838.0,-17515.0,161340.0,35195.00,-22.68,38.0,X-MC,8.97,64.0,-0.35,1.16,4.95,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,1.89,0.15,20.56,20.74,41834.0,302.0,203472.0,446.0,31.15,53.0,M-MC,4.55,189.0,0.01,1.46,10.78,XY25,NTT,FINANCE
84,WIPRO,0.34,-0.28,72.75,72.27,122929.0,-473.0,168975.0,420.0,-14.25,47.0,M-LC,2.61,101.0,-0.00,1.22,8.61,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.58,-6.08,121.38,107.91,168306.0,-8984.0,138660.0,1641.55,-23.39,49.0,M-SC,11.26,220.0,-0.05,1.00,4.32,OX40N,ATH,CABLES
72,TATAELXSI,0.44,-21.25,67.66,32.03,70242.0,-28019.0,103816.0,9161.00,-14.86,52.0,H-SC,6.50,157.0,-0.40,0.75,16.28,OX40N,NTT,IT
66,SIS,-0.38,-22.57,56.61,21.26,48669.0,-25059.0,85973.0,528.00,2030.57,56.0,H-SC,5.00,163.0,-0.51,0.62,16.16,OX40N,NTT,MISC
48,KANSAINER,-2.59,-23.87,49.06,13.47,100715.0,-64377.0,205290.0,340.00,-68.70,46.0,H-SC,5.12,159.0,-0.64,1.48,4.38,XY24,NTT,PAINTS
40,INDIGOPNTS,0.17,-26.53,36.14,0.02,46346.0,-46318.0,128241.0,1408.00,95.31,31.0,M-SC,5.24,240.0,-1.00,0.92,11.55,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,1.89,0.15,20.56,20.74,41834.0,302.0,203472.0,446.00,31.15,53.0,M-MC,4.55,189.0,0.01,1.46,10.78,XY25,NTT,FINANCE
18,COALINDIA,0.26,4.41,14.19,19.23,22293.0,6642.0,157102.0,484.83,22.81,52.0,L-LC,12.49,182.0,0.30,1.13,26.63,XY25,ATH,MINING
1,ABB,4.58,4.92,38.74,45.57,106337.0,12869.0,274488.0,7934.00,-32.57,71.0,H-MC,6.04,121.0,0.12,1.97,21.89,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,4.58,4.92,38.74,45.57,106337.0,12869.0,274488.0,7934.00,-32.57,71.0,H-MC,6.04,121.0,0.12,1.97,21.89,AR,NTT,ELECTRICAL
84,WIPRO,0.34,-0.28,72.75,72.27,122929.0,-473.0,168975.0,420.00,-14.25,47.0,M-LC,2.61,101.0,-0.00,1.22,8.61,XR,NTT,IT
39,INDIAMART,2.69,-2.47,111.97,106.72,134682.0,-3052.0,120284.0,4810.62,-51.77,57.0,H-SC,8.90,138.0,-0.02,0.87,20.32,AR,ATH,MISC
26,FINCABLES,2.58,-6.08,121.38,107.91,168306.0,-8984.0,138660.0,1641.55,-23.39,49.0,M-SC,11.26,220.0,-0.05,1.00,4.32,OX40N,ATH,CABLES
85,ZYDUSLIFE,2.35,-5.06,42.85,35.63,85262.0,-10597.0,198977.0,1286.17,-18.51,52.0,H-MC,5.83,119.0,-0.12,1.43,10.77,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,-1.08,-16.04,44.06,20.95,93132.0,-40392.0,211376.0,3460.25,-16.94,23.0,X-LC,10.58,36.0,-0.43,1.52,13.88,X40,ATH,PAINTS
44,ITC,0.68,-22.34,42.65,10.78,79596.0,-53693.0,186625.0,452.00,-53.82,24.0,X-LC,18.39,1.0,-0.67,1.34,0.68,X40,NTT,FMCG
12,BATAINDIA,0.28,-44.14,142.05,35.20,102097.0,-56796.0,71874.0,2096.00,-5.70,29.0,X-SC,20.43,93.0,-0.56,0.52,2.14,X40,NTT,FOOTWEAR
81,VBL,-4.84,-10.36,51.36,35.67,145406.0,-32730.0,283112.0,671.64,-21.08,29.0,X-LC,1.16,5.0,-0.23,2.04,2.06,X40N,ATH,FMCG
13,BERGEPAINT,1.68,-15.56,43.46,21.14,83430.0,-35369.0,191970.0,680.00,-21.59,30.0,X-MC,7.77,75.0,-0.42,1.38,2.40,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,1.74,-11.33,33.70,18.55,113023.0,-42858.0,335379.0,4311.59,-23.62,57.0,X-LC,0.64,3.0,-0.38,2.41,14.10,X40,ATH,IT
81,VBL,-4.84,-10.36,51.36,35.67,145406.0,-32730.0,283112.0,671.64,-21.08,29.0,X-LC,1.16,5.0,-0.23,2.04,2.06,X40N,ATH,FMCG
5,ANGELONE,7.86,6.06,17.10,24.20,55361.0,18500.0,323750.0,3033.00,50.86,54.0,X-SC,1.99,99.0,0.33,2.33,35.59,X40N,NTT,FINANCE
34,HINDUNILVR,1.01,-7.38,22.72,13.67,54096.0,-18963.0,238100.0,2922.00,-14.28,52.0,X-LC,2.26,9.0,-0.35,1.71,12.40,XY25,NTT,FMCG
21,DABUR,0.42,-2.03,46.68,43.70,113448.0,-5034.0,243034.0,735.00,-10.16,42.0,X-MC,2.75,73.0,-0.04,1.75,12.72,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,0.68,-22.34,42.65,10.78,79596.0,-53693.0,186625.0,452.00,-53.82,24.0,X-LC,18.39,1.0,-0.67,1.34,0.68,X40,NTT,FMCG
53,PGHH,0.44,-11.29,51.26,34.19,97099.0,-24096.0,189424.0,17907.65,-37.61,32.0,X-MC,8.05,60.0,-0.25,1.36,0.72,X40,ATH,FMCG
81,VBL,-4.84,-10.36,51.36,35.67,145406.0,-32730.0,283112.0,671.64,-21.08,29.0,X-LC,1.16,5.0,-0.23,2.04,2.06,X40N,ATH,FMCG
12,BATAINDIA,0.28,-44.14,142.05,35.20,102097.0,-56796.0,71874.0,2096.00,-5.70,29.0,X-SC,20.43,93.0,-0.56,0.52,2.14,X40,NTT,FOOTWEAR
13,BERGEPAINT,1.68,-15.56,43.46,21.14,83430.0,-35369.0,191970.0,680.00,-21.59,30.0,X-MC,7.77,75.0,-0.42,1.38,2.40,XY24,NTT,PAINTS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.96,-28.91,100.93,42.84,46643.0,-18793.0,46213.0,424.00,-53.72,55.0,X-SC,21.68,83.0,-0.40,0.33,8.63,XY24,NTT,MISC
12,BATAINDIA,0.28,-44.14,142.05,35.20,102097.0,-56796.0,71874.0,2096.00,-5.70,29.0,X-SC,20.43,93.0,-0.56,0.52,2.14,X40,NTT,FOOTWEAR
58,RELAXO,1.92,-49.58,202.08,52.31,147898.0,-71972.0,73188.0,1176.00,-46.28,51.0,X-SC,11.58,92.0,-0.49,0.53,8.83,X40N,NTT,FOOTWEAR
51,MEDANTA,-2.54,-7.22,33.73,24.08,39730.0,-9163.0,117787.0,1486.00,-14.79,48.0,X-SC,10.32,91.0,-0.23,0.85,7.88,XY24,NTT,HEALTHCARE
35,HONAUT,0.13,-17.35,73.40,43.32,98811.0,-28252.0,134620.0,58357.33,-28.92,54.0,X-SC,8.83,90.0,-0.29,0.97,9.30,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,0.68,-22.34,42.65,10.78,79596.0,-53693.0,186625.0,452.00,-53.82,24.0,X-LC,18.39,1.0,-0.67,1.34,0.68,X40,NTT,FMCG
75,TMPV,2.69,-23.62,61.01,22.98,105265.0,-53361.0,172537.0,600.00,-20.63,64.0,X-LC,4.41,2.0,-0.51,1.24,10.30,XY24,NTT,AUTO
73,TCS,1.74,-11.33,33.70,18.55,113023.0,-42858.0,335379.0,4311.59,-23.62,57.0,X-LC,0.64,3.0,-0.38,2.41,14.10,X40,ATH,IT
81,VBL,-4.84,-10.36,51.36,35.67,145406.0,-32730.0,283112.0,671.64,-21.08,29.0,X-LC,1.16,5.0,-0.23,2.04,2.06,X40N,ATH,FMCG
42,INFY,2.03,13.18,18.62,34.24,65936.0,41224.0,354112.0,1972.00,-11.70,56.0,X-LC,7.23,6.0,0.63,2.55,22.55,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1.48,-31.80,94.87,32.89,49950.0,-24556.0,52651.0,1800.00,-749.97,60.0,L-MC,11.22,259.0,-0.49,0.38,45.00,XR,NTT,BANKS
67,SONACOMS,5.60,-8.75,52.44,39.09,48404.0,-8855.0,92304.0,804.02,-28.63,72.0,M-SC,2.37,227.0,-0.18,0.66,30.58,AR,ATH,AUTO
14,BSOFT,4.44,-19.42,87.13,50.79,98749.0,-27318.0,113335.0,831.70,3.21,61.0,H-SC,6.15,171.0,-0.28,0.82,31.96,XR,ATH,IT
79,VAIBHAVGBL,11.52,-23.12,109.12,60.77,153330.0,-42260.0,140515.0,521.00,61.98,64.0,H-SC,4.46,158.0,-0.28,1.01,27.39,XR,NTT,JEWELLERY
18,COALINDIA,0.26,4.41,14.19,19.23,22293.0,6642.0,157102.0,484.83,22.81,52.0,L-LC,12.49,182.0,0.30,1.13,26.63,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,4.58,4.92,38.74,45.57,106337.0,12869.0,274488.0,7934.00,-32.57,71.0,H-MC,6.04,121.0,0.12,1.97,21.89,AR,NTT,ELECTRICAL
67,SONACOMS,5.60,-8.75,52.44,39.09,48404.0,-8855.0,92304.0,804.02,-28.63,72.0,M-SC,2.37,227.0,-0.18,0.66,30.58,AR,ATH,AUTO
5,ANGELONE,7.86,6.06,17.10,24.20,55361.0,18500.0,323750.0,3033.00,50.86,54.0,X-SC,1.99,99.0,0.33,2.33,35.59,X40N,NTT,FINANCE
27,GILLETTE,1.51,4.14,27.15,32.42,71786.0,10509.0,264405.0,11206.78,-14.43,66.0,X-SC,12.38,87.0,0.15,1.90,19.86,X40,ATH,FMCG
79,VAIBHAVGBL,11.52,-23.12,109.12,60.77,153330.0,-42260.0,140515.0,521.00,61.98,64.0,H-SC,4.46,158.0,-0.28,1.01,27.39,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.73
1,25,44.72
2,50,76.62


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.27
MC    30.52
LC    24.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.33
X40      23.82
X40N     14.99
AR        9.00
XR        8.86
XY25      8.64
OX40N     7.41
SR        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.84
H-SC    23.03
X-LC    20.69
M-SC    11.12
X-SC    10.37
H-MC     4.84
M-MC     1.46
M-LC     1.22
H-LC     1.15
L-LC     1.13
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.90,-14.30,53.26
FINANCE,13.35,-12.55,56.87
IT,12.95,-18.89,78.28
MISC,6.98,-33.76,88.48
ELECTRICAL,6.08,-10.92,51.39
PAINTS,5.30,-25.30,43.92
INSURANCE,4.81,-1.37,36.06
PHARMA,3.98,-6.10,39.50
AUTO,2.95,-27.61,71.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3310719.0,21
AR,1319157.0,10
XR,1260813.0,12
X40,1196957.0,15
X40N,1038787.0,10
OX40N,761297.0,10
XY25,417735.0,6
SR,291051.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3592170.0,24
X-MC,1717790.0,16
M-SC,1512157.0,15
X-LC,1015062.0,12
X-SC,857557.0,9
H-MC,405252.0,3
L-SC,186260.0,2
M-LC,122929.0,1
H-LC,73262.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1234353.0      6
           AR         931861.0      5
M-SC       XY24       861656.0      6
H-SC       XR         810690.0      7
X-MC       X40        580277.0      7
           XY24       482062.0      3
X-LC       X40        442797.0      6
X-MC       X40N       421827.0      4
X-SC       X40N       374697.0      4
M-SC       OX40N      344731.0      5
H-SC       OX40N      324215.0      4
X-SC       XY24       308977.0      3
H-SC       SR         291051.0      2
X-LC       X40N       242263.0      2
X-MC       XY25       233624.0      2
H-MC       XY24       213653.0      1
X-LC       XY24       210018.0      2
H-MC       AR         191599.0      2
M-SC       XR         183335.0      2
X-SC       X40        173883.0      2
M-LC       XR         122929.0      1
M-SC       AR         122435.0      2
X-LC       XY25       119984.0      2
L-SC       XR          93909.0      1
           OX40N       92351.0      1
H-LC       AR          73262.0      1
L-MC       XR          49950.0      1
M-MC       XY25        41834.0      1
L-LC       XY25        22293.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
